<a href="https://colab.research.google.com/github/itsAkarsh/Data-Reconciliation-Tool-/blob/main/DRT(StableVersion).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title DRT - Data Reconciliation Tool {display-mode: "form"}
!pip install fuzzywuzzy
import pandas as pd
from google.colab import files
from IPython.display import display
import ipywidgets as widgets
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def upload_excel_files():
    print("Please upload the source Excel file:")
    source_file = files.upload()
    print("Please upload the destination Excel file:")
    destination_file = files.upload()
    return pd.read_excel(next(iter(source_file.values()))), pd.read_excel(next(iter(destination_file.values())))

def display_number_of_keys_dropdown(column_names_source):
    number_of_keys_dropdown = widgets.Dropdown(options=list(range(1, len(column_names_source)+1)), description="Keys:")
    display(number_of_keys_dropdown)
    button = widgets.Button(description="Done")
    display(button)

    def on_button_clicked(b):
        global number_of_keys
        number_of_keys = number_of_keys_dropdown.value
        print(f"Number of Keys selected: {number_of_keys}")
        select_keys(column_names_source)

    button.on_click(on_button_clicked)

def select_keys(column_names_source):
    selected_keys_dropdowns = []
    for i in range(number_of_keys):
        key_dropdown = widgets.Dropdown(options=column_names_source, description=f"Key {i+1}:")
        selected_keys_dropdowns.append(key_dropdown)
    display(widgets.VBox(selected_keys_dropdowns))
    save_button = widgets.Button(description="Save Key")
    display(save_button)

    def on_save_button_clicked(b):
        global selected_keys
        selected_keys = [key_dropdown.value for key_dropdown in selected_keys_dropdowns]
        print(f"Selected Keys: {selected_keys}")
        select_source_columns(column_names_source)

    save_button.on_click(on_save_button_clicked)

def select_source_columns(column_names_source):
    checkboxes_source = [widgets.Checkbox(description=col) for col in column_names_source]
    button = widgets.Button(description="Done")
    selected_source_column = []

    def on_button_clicked(b):
        nonlocal selected_source_column
        selected_source_column = [checkbox.description for checkbox in checkboxes_source if checkbox.value]
        map_destination_columns(selected_source_column)

    button.on_click(on_button_clicked)
    print("Select the source column for recon: ")
    display(widgets.VBox(checkboxes_source + [button]))

def map_destination_columns(selected_source_column):
    mapped_data = pd.DataFrame(columns=['Source', 'Destination'])
    checkbox_map = {}

    for item in selected_source_column:
        checkboxes = [widgets.Checkbox(description=option) for option in column_names_destination]
        checkbox_map[item] = checkboxes

    def on_checkbox_change(change):
        nonlocal mapped_data
        mapped_data.drop(mapped_data.index, inplace=True)
        data = []
        for key, checkboxes in checkbox_map.items():
            for checkbox in checkboxes:
                if checkbox.value:
                    data.append((key, checkbox.description))
        mapped_data = pd.DataFrame(data, columns=['Source', 'Destination'])

    for item, checkboxes in checkbox_map.items():
        print(f"Mapped the Destination column to Source Column-{item}:")
        for checkbox in checkboxes:
            display(checkbox)
            checkbox.observe(on_checkbox_change, names='value')

    done_button = widgets.Button(description="Done")
    display(done_button)

    def on_done_button_clicked(b):
        print("Mapped Data:")
        display(mapped_data)
        merge_and_compare_data(mapped_data)

    done_button.on_click(on_done_button_clicked)

def fuzzy_match(source, destination):
    if isinstance(source, str) and isinstance(destination, str):
        similarity_ratio = fuzz.token_set_ratio(source.lower(), destination.lower())
        return similarity_ratio >= 50
    else:
        return False

def merge_and_compare_data(mapped_data, output_file='comparison_report.xlsx'):
    merged_data = pd.merge(source_df, destination_df, how='left', left_on=selected_keys, right_on=[mapped_data.loc[mapped_data['Source'] == key, 'Destination'].iloc[0] for key in selected_keys])
    merged_df = pd.DataFrame(merged_data)
    mapped_df = pd.DataFrame(mapped_data)
    report_df = pd.DataFrame()

    for index, row in mapped_df.iterrows():
        source_col = row['Source']
        destination_col = row['Destination']
        if source_col in merged_df.columns and destination_col in merged_df.columns:
            comparison_result = merged_df.apply(lambda x: fuzzy_match(str(x[source_col]), str(x[destination_col])), axis=1)
            report_df[f'{source_col} (Source)'] = merged_df[source_col]
            report_df[f'{destination_col} (Destination)'] = merged_df[destination_col]
            report_df[f'Comparison Result ({source_col} vs {destination_col})'] = ['Matching' if val else 'Not Matching' for val in comparison_result]
        else:
            print(f"Columns {source_col} or {destination_col} not found in merged_df")

    report_df = report_df.dropna(axis=1, how='all')
    report_df.to_excel(output_file, index=False)
    print(f"Comparison report saved to '{output_file}'")

source_df, destination_df = upload_excel_files()
column_names_source = source_df.columns.tolist()
column_names_destination = destination_df.columns.tolist()

display_number_of_keys_dropdown(column_names_source)


Please upload the source Excel file:


Saving Admin_FlatFile_Shiftwise.xlsx to Admin_FlatFile_Shiftwise (2).xlsx
Please upload the destination Excel file:


Saving Otheruser shiftwise.xlsx to Otheruser shiftwise.xlsx


<ipython-input-13-2bd27641f77f>:15: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  return pd.read_excel(next(iter(source_file.values()))), pd.read_excel(next(iter(destination_file.values())))
<ipython-input-13-2bd27641f77f>:15: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  return pd.read_excel(next(iter(source_file.values()))), pd.read_excel(next(iter(destination_file.values())))


Dropdown(description='Keys:', options=(1, 2, 3, 4, 5, 6), value=1)

...

Checkbox(value=False, description='Project Code')

Button(description='Done', style=ButtonStyle())

Mapped Data:


,Source,Destination
0,Source Country,Country
1,Source City,City
2,Source Building,Building
3,Source Floor/Block/Tower,Floor/Block/Tower
4,Source Desk ID,Desk ID
5,Source Project Code,Project Code


Comparison report saved to 'comparison_report.xlsx'
